In [1]:
import pandas as pd
import re

# **Importing Datasets**

In [2]:
df = pd.read_csv('data_turnbachoax_dirty.csv')

In [3]:
df.head()

,headline,time,content,link,hoax
0,[SALAH] Konvoi Bantuan Palestina Dirampas oleh...,"November 25, 2023",Hasil periksa fakta Vendra PanjiDalam video as...,https://turnbackhoax.id/2023/11/25/salah-konvo...,1
1,[SALAH] Video Polisi Israel Mencekik Anak Pale...,"November 23, 2023",Hasil periksa fakta Moch. MarcellodiansyahFakt...,https://turnbackhoax.id/2023/11/23/salah-video...,1
2,[SALAH] Video Tentara Israel Memasang Bendera ...,"November 23, 2023",Hasil periksa fakta Evarizma Zahra. Konten yan...,https://turnbackhoax.id/2023/11/23/salah-video...,1
3,[SALAH] Perusahaan Israel Pepsi Ubah Kemasan B...,"November 23, 2023",Hasil periksa fakta Moch. MarcellodiansyahFakt...,https://turnbackhoax.id/2023/11/23/salah-perus...,1
4,[SALAH] Cristiano Ronaldo Lempar Mic Wartawan ...,"November 22, 2023",Hasil periksa fakta Vendra PanjiCR7 melempar m...,https://turnbackhoax.id/2023/11/22/salah-crist...,1


**Finding Missing Value**

In [4]:
df.isna().sum()

headline    0
time        0
content     0
link        0
hoax        0
dtype: int64

**Implementasi Regex untuk membersihkan data yang diapit oleh kurung siku**

In [5]:
df['headline'] = df['headline'].str.replace(r'\[.*?\]|\(.*?\)', '', regex=True)

**Removing space at the begining and the end of sentencec**

In [6]:
headline_strip = df['headline']
list_new_headline = []
print(len(headline_strip))
for i in headline_strip:
  list_new_headline.append(i.strip())
print(len(list_new_headline))

322
322


In [7]:
df['headline'] = list_new_headline

In [8]:
df[['headline']]

,headline
0,Konvoi Bantuan Palestina Dirampas oleh Israel
1,Video Polisi Israel Mencekik Anak Palestina di...
2,Video Tentara Israel Memasang Bendera Israel d...
3,Perusahaan Israel Pepsi Ubah Kemasan Bertulisa...
4,Cristiano Ronaldo Lempar Mic Wartawan Israel k...
...,...
317,Menteri Agama Yaqut Cholil Berasal dari Kelomp...
318,Tulisan Ridwan Kamil : “NIAT BUSUK KUBU SEBELA...
319,Video “BERITA TERBARU ~ RONALDO SUMBANGKAN GAJ...
320,Video “Kapal oleng di pelabuhan padangbae”


**Cleaning content datasets which has no meaning**

In [9]:
df['content'].iloc[0]

'Hasil periksa fakta Vendra PanjiDalam video aslinya tersebut tidak berisi penjelasan adanya peristiwa perampasan yang dilakukan oleh Israel. Truk-truk tersebut berhenti karena sedang dalam proses untuk melewati perbatasan Rafah, yaitu perbatasan antara Mesir dengan Gaza.===========[KATEGORI]: Konten dimanipulasi===========[SUMBER]: Tiktok https://archive.cob.web.id/archive/1700676627.275035/index.html\xa0===========[NARASI]: “Ini Gambaran Konvoi Bantuan Untuk Palestina Yang Di Rampas ISRAELSebarkan Agar Dunia Tau Betapa K3J@m ISRAEL”===========[PENJELASAN]:\xa0Sebuah video di Tiktok memperlihatkan banyak sekali truk yang terparkir memanjang, video itu juga disertai dengan narasi jika truk-truk tersebut merupakan bantuan untuk Palestina yang sedang dirampas Israel.Namun, setelah ditelusuri rupanya potongan video tersebut berasal dari sebuah media asal Mesir bernama Al Nahar, video itu diunggah di Facebook pada 31 Mei 2021 lalu. Video juga disertai dengan caption berbahasa Arab yang jik

In [10]:
df['content'] = df['content'].str.replace('=', '')

In [11]:
df['content'] = df['content'].str.replace(r'\[.*?\]|\(.*?\)', '', regex=True)

In [12]:
df['content'].iloc[0]

'Hasil periksa fakta Vendra PanjiDalam video aslinya tersebut tidak berisi penjelasan adanya peristiwa perampasan yang dilakukan oleh Israel. Truk-truk tersebut berhenti karena sedang dalam proses untuk melewati perbatasan Rafah, yaitu perbatasan antara Mesir dengan Gaza.: Konten dimanipulasi: Tiktok https://archive.cob.web.id/archive/1700676627.275035/index.html\xa0: “Ini Gambaran Konvoi Bantuan Untuk Palestina Yang Di Rampas ISRAELSebarkan Agar Dunia Tau Betapa K3J@m ISRAEL”:\xa0Sebuah video di Tiktok memperlihatkan banyak sekali truk yang terparkir memanjang, video itu juga disertai dengan narasi jika truk-truk tersebut merupakan bantuan untuk Palestina yang sedang dirampas Israel.Namun, setelah ditelusuri rupanya potongan video tersebut berasal dari sebuah media asal Mesir bernama Al Nahar, video itu diunggah di Facebook pada 31 Mei 2021 lalu. Video juga disertai dengan caption berbahasa Arab yang jika diterjemahkan artinya adalah ““For the third time.. #Egypt sends the largest con

**Implementing Stopwords to remove the common words**

In [13]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [14]:
nltk.download('stopwords')
nltk.download('punkt')
# nltk.download('indonesian')
# stop_words = set(stopwords.words('indonesian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [15]:
stop_words_file = 'stopwords_indonesian.txt'
with open(stop_words_file, 'r') as file:
    stop_words = set(file.read().splitlines())

print(len(stop_words))

757


In [16]:
cleaned_content = []
for index in df['content']:
  word_tokens = word_tokenize(index)
  filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
  cleaned_content.append(' '.join(filtered_sentence))

print(len(cleaned_content))

322


In [17]:
df['content'] = cleaned_content

In [18]:
df['content'].iloc[29]

'Hasil periksa fakta Mochamad MarcellFaktanya video diambil perayaan kembang api Aljazair klub bola CR Belouizdad memenangkan kejuaraan Aljazair 2020 . Selengkapnya penjelasan . Kategori : Konteks Salah Sumber : Twitterhttps : //ghostarchive.org/archive/aHJCM Narasi : “ Hell on Earth Gaza . War is a bad thing for humanity . Innocent people do not deserve to live in this chaos . Stop the war ! # Israel # Palestine # Gaza # Hamas ” Terjemahan : “ Neraka Bumi Gaza . Perang buruk umat manusia . Orang bersalah hidup kekacauan . Hentikan perang ! # Israel # Palestina # Gaza # Hamas ” Penjelasan : Video Twitter kota terbakar malam diklaim Gaza akibat serangan Israel jalur Gaza.Setelah ditelusuri klaim salah , faktanya video diambil Aljazair perayaan kembang api klub bola CR Belouizdad memenangkan kejuaraan Aljazair 2020 . Wartawan AFP mengonfirmasi rekaman tersebar internet kali 27 September 2023 , seminggu Hamas menyerang Israel 7 Oktober 2023.Dengan , video Gaza terbakar diserang Israel kat

Splitting times data into three columns, date, month, year


In [19]:
from datetime import datetime

In [20]:
df['time'].iloc[317]

'Mei 5, 2021'

In [21]:
df['date'] = ''
df['month'] = ''
df['year'] = ''

def split_date(date_string):
    match = re.match(r'(\w+) (\d+), (\d+)', date_string)
    if match:
        month, date, year = match.groups()
        return month, int(date), int(year)
    return None, None, None

for index, row in df[['time']].iterrows():
    month, date, year = split_date(row['time'])
    if all([month, date, year]):
        df.at[index, 'month'] = month
        df.at[index, 'date'] = date
        df.at[index, 'year'] = year

In [22]:
df['month'].unique()

array(['November', 'Oktober', 'September', 'Juni', 'Mei', 'April',
       'Maret', 'Januari', 'Desember', 'Februari', 'Juli', 'Agustus'],
      dtype=object)

In [25]:
df['year'].unique()

array([2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015], dtype=object)

In [26]:
df['date'].nunique()

31

In [27]:
df.isna().sum()

headline    0
time        0
content     0
link        0
hoax        0
date        0
month       0
year        0
dtype: int64

Turning the mounth data into correct format

In [32]:
df['month'] = df['month'].map({
    'November':'November',
    'Oktober':'October',
    'September':'September',
    'Juni':'June',
    'Mei':'May',
    'April':'April',
    'Maret':'March',
    'Januari':'January',
    'Desember':'December',
    'Februari':'February',
    'Juli':'July',
    'Agustus':'August'
})

In [33]:
df['month'].unique()

array(['November', 'October', 'September', 'June', 'May', 'April',
       'March', 'January', 'December', 'February', 'July', 'August'],
      dtype=object)

In [35]:
df.head()

,headline,time,content,link,hoax,date,month,year
0,Konvoi Bantuan Palestina Dirampas oleh Israel,"November 25, 2023",Hasil periksa fakta Vendra PanjiDalam video as...,https://turnbackhoax.id/2023/11/25/salah-konvo...,1,25,November,2023
1,Video Polisi Israel Mencekik Anak Palestina di...,"November 23, 2023",Hasil periksa fakta Moch . MarcellodiansyahFak...,https://turnbackhoax.id/2023/11/23/salah-video...,1,23,November,2023
2,Video Tentara Israel Memasang Bendera Israel d...,"November 23, 2023",Hasil periksa fakta Evarizma Zahra . Konten me...,https://turnbackhoax.id/2023/11/23/salah-video...,1,23,November,2023
3,Perusahaan Israel Pepsi Ubah Kemasan Bertulisa...,"November 23, 2023",Hasil periksa fakta Moch . MarcellodiansyahFak...,https://turnbackhoax.id/2023/11/23/salah-perus...,1,23,November,2023
4,Cristiano Ronaldo Lempar Mic Wartawan Israel k...,"November 22, 2023",Hasil periksa fakta Vendra PanjiCR7 melempar m...,https://turnbackhoax.id/2023/11/22/salah-crist...,1,22,November,2023


In [36]:
df = df[['headline','date','month','year','content','link','hoax']]

In [37]:
df.to_csv('data_turnbackhoax_clean.csv', index = False)